In [1]:
import numpy as np
import cv2


IP_file = 'left_output-1.avi'
vid = cv2.VideoCapture(IP_file)
template_img = cv2.imread('Template-1.png')
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.8
color = (0, 0, 255)
thickness = 2


while vid.isOpened() :
    ret, query_img = vid.read()
    query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
    if ret :
        sift = cv2.SIFT_create()
        bf = cv2.BFMatcher()
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)
        
        good_matches = list()
        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                
        
        if len(good_matches) > 10 :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 2.5)
            h, w = template_img.shape[:2]
            


            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)
            x = [np.int32(transformed_box)]

            detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,255,0), 3, cv2.LINE_AA)
            
            w_max = query_img.shape[1] / 2
            h_max = query_img.shape[0] / 2


            x1 = int(x[0][0][0][0]) #X topleft
            y1 = int(x[0][0][0][1]) #y topleft
            x2 = int(x[0][1][0][0]) #x botleft
            y2 = int(x[0][1][0][1]) #y botleft
            z = y2 - y1

            X_Current = x1 - w_max
            Y_Current = h_max - y1
            Z_Current = 200 - z

            str1 = '('+"{:.0f}".format(X_Current) + ',' +"{:.0f}".format(Y_Current) + ',' +"{:.0f}".format(Z_Current)+ ')'

            detected_img2 = cv2.putText(detected_img,str1, (x1,y1), font,fontScale, color, thickness, cv2.LINE_AA)
            cv2.imshow('Video frame', detected_img)

        if cv2.waitKey(int(1000/24)) & 0xFF == ord('q') :
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()